In [2]:
from agol_restapi_tools import token_generation
from agol_restapi_tools import agol_table_to_pd
from agol_restapi_tools import oid_field
from agol_table_pull import survey_data

import requests
import pandas as pd
import datetime
from pytz import timezone
import pytz
import json

import warnings
warnings.simplefilter('ignore')

#Set Alaska Timezone
#alaska_tz = pytz.timezone('US/Alaska')


#Survey URL
survey_url = 'https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/APEX_FORM_VIEW/FeatureServer'

#Starts URL
starts_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/Start_Points/FeatureServer"

#Ends URL
ends_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/End_Points/FeatureServer"

#Projects URL
projects_url= "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/CY2023_Project_Polygons_notcleaned/FeatureServer"

#Project Points URL
points_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/CY2023_Project_Points/FeatureServer"

#Alert Logbook URL
alert_log_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/emailed_managers_log/FeatureServer"

#Update Logbook URL
update_log_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/update_log/FeatureServer"

#Alaska 511 Management URL
management_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/AK_511_Management/FeatureServer"

#Set Token
token = token_generation(username = "AKDOT_COMMUNICATIONS", password = 'Skisnowbird1')




#TEST SURVEY
survey_url = 'https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/survey_test/FeatureServer'

In [ ]:
survey_applyEdits = f'{survey_url}/0/applyEdits'

### CREATE FUNCTION TO CHECK UPDATE STATUS

In [3]:
def check_update_status(creationDate):

    #Set Alaska Timezone
    alaska_tz = pytz.timezone('US/Alaska')

    #Save Current Date and Time for Comparison
    today = datetime.datetime.now(alaska_tz).date()

    #Find the Date for This Week's Monday
    this_monday = today - datetime.timedelta(days=today.weekday()) + datetime.timedelta(days = 0)

    #Find the Date for This Week's Sunday
    this_sunday = this_monday + datetime.timedelta(days = 6)

    #Find the date that is 4 Weeks Back from the Sunday
    four_weeks_back = this_monday - datetime.timedelta(days = 28)



    #Check CreationDate Status Against Dates
    status = ''

    if creationDate < four_weeks_back:
        status = "Out of Date"

    elif creationDate >= four_weeks_back and creationDate < this_monday:
        status = 'Awaiting Update'

    elif creationDate >= this_monday and creationDate <= this_sunday:
        status = 'Updated'


    return status

### PULL THE SURVEY DATA

In [7]:
def update_status(service_url, layer, token, max_date, entry):

    ########################### GATHER OID FIELD AND SET CREATIONDATE #########################

    #Locate the ObjectID Field Name of the Servie Layer
    oid_field = oid_field(service_url, layer, token)

    #Set the CreationDate
    creationDate = entry['CreationDate']



    ########################## CHECK STATUS OF THE CREATIONDATE #################################

    #If the submission Creationdate not the most recent, set status to blank and upload to AGOL
    if row['CreationDate'] != max_date:
        status = ''


    #If the submission CreationDate is the most recent entry, set status based on timeframes and upload to AGOL
    elif row['CreationDate'] == max_date:
        status = check_update_status(creationDate)  




    ############################### UPLOAD STATUS TO AGOL SURVEY TABLE ###############################

    if status != None:

        #Create Update Payload
        payload = [
            {'attributes': {
                oid_field: entry[survey_objid_field],  #Reference the OID Field to Update
                'Weekly_Status_Update': ''
                }
            }]
        

        #Set Update Parameters
        params = {
            'f':'json',
            'token': token,
            'updates': f"{payload}"
            }

        try:
            #Send Update to AGOL
            response = requests.post(url = f'{survey_url}/0/applyEdits', params = params).json()
            print(response)

        except:
            #ADD ERROR CATCH AND LOG IF FAIL
            pass

In [8]:
survey = survey_data(survey_url, 0, token)
survey_uids = list(survey['UID'].unique())
survey_objid_field = oid_field(survey_url, 0, token)

check = pd.DataFrame()

for uid in survey_uids:

    #CHECK: UID Exists in Table
    if 'UID' in survey.columns:
        uid_sel = survey[survey['UID'] == uid]  #Select all project entries in survey table

        #CHECK: CreationDate and Weekly_Update_Status in uid_selection
        if "CreationDate" in uid_sel.columns and "Weekly_Update_Status" in uid_sel.columns:
            uid_sel['CreationDate'] = pd.to_datetime(uid_sel['CreationDate'], unit = "ms")  #Convert CreationDate from UTC ms to timestamp
            max_date = uid_sel['CreationDate'].max()  #Find the maximum date in each of the project selections

            #Iterate Through Project Selections and Apply Update Status
            for index, row in uid_sel.iterrows():
                update_status(survey_url, 0, token, max_date, row)

TypeError: 'str' object is not callable